In [1]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchinfo import summary
import torch.optim as optim
import datetime
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from metrics import deviation_metric
import catboost as cb
from metrics import deviation_metric_one_sample

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
df = pd.read_csv('./Data/train_parse3.csv')
X_train, X_test = df[df["price_type"] == 0].drop(columns=["target"]), df[df["price_type"] != 0].drop(columns=["target"])
Y_train, Y_test = df[df["price_type"] == 0]["target"], df[df["price_type"] != 0]["target"]
scaler = StandardScaler()

X_train.iloc[:,:] = scaler.fit_transform(X_train.iloc[:,:])
X_test.iloc[:,:] = scaler.transform(X_test.iloc[:,:])

# scaler2 = StandardScaler()
# Y_train = scaler2.fit_transform(Y_train.values.reshape(-1,1))
# Y_test = scaler2.transform(Y_test.values.reshape(-1,1))

Y_train

0         139937.500000
1          60410.714286
2          45164.761264
3          28805.263158
4          13222.591362
              ...      
275294     25450.669915
275295     43525.000000
275296     28727.272727
275297     79068.245559
275298     41160.714286
Name: target, Length: 275299, dtype: float64

In [3]:
model = cb.CatBoostRegressor(
    iterations=100,
    learning_rate=0.05,
    eval_metric=R,
    task_type="GPU",
    devices='0:1'
)
model.fit(
    X_train,
    Y_train,
    verbose=False,
    plot=True
)

NameError: name 'R' is not defined

In [ ]:
aa = model.predict(
    X_test
)
print(deviation_metric(Y_test, aa))

In [7]:
#Настин Катбуст
class RmseObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)

        result = []
        for index in range(len(targets)):
            der1 = targets[index] - approxes[index]
            der2 = -1
            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]
            result.append((der1, der2))
        return result

class RmseMetric(object):
    def get_final_error(self, error, weight):
        return np.sqrt(error / (weight + 1e-38))
    def is_max_optimal(self):
        return False
    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        approx = approxes[0]
        error_sum = 0.0
        weight_sum = 0.0
        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error_sum += w * (deviation_metric_one_sample(target[i], approx[i])**2)
        return error_sum, weight_sum

model = cb.CatBoostRegressor(iterations=16, depth=15, learning_rate=0.1, loss_function=RmseObjective(), verbose=True, eval_metric = RmseMetric())

In [8]:
model.fit(
    X_train,
    Y_train,
    verbose=True,
    plot=True
)

C:\Users\gto_n\.conda\envs\raifhack_2021\lib\site-packages\catboost\core.py:1913: UserWarning: Failed to import numba for optimizing custom metrics and objectives
  _check_train_params(params)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Got unsafe target value = 1.63931e+06 at object #576 of dataset learn


0:	learn: 9.5805716	total: 3.66s	remaining: 55s
1:	learn: 9.1928775	total: 7.32s	remaining: 51.3s
2:	learn: 8.7804832	total: 11s	remaining: 47.7s
3:	learn: 8.3017511	total: 14.7s	remaining: 44.1s
4:	learn: 7.7753389	total: 18.5s	remaining: 40.7s
5:	learn: 7.2815133	total: 22.3s	remaining: 37.1s
6:	learn: 6.8509196	total: 26s	remaining: 33.4s
7:	learn: 6.5018802	total: 29.8s	remaining: 29.8s
8:	learn: 6.2138723	total: 33.6s	remaining: 26.2s
9:	learn: 6.0053069	total: 37.5s	remaining: 22.5s
10:	learn: 5.8344883	total: 41.5s	remaining: 18.9s
11:	learn: 5.7052452	total: 45.4s	remaining: 15.1s
12:	learn: 5.5942133	total: 49.4s	remaining: 11.4s
13:	learn: 5.5287257	total: 53.2s	remaining: 7.6s
14:	learn: 5.4604700	total: 57.1s	remaining: 3.81s
15:	learn: 5.4221090	total: 1m 1s	remaining: 0us


In [9]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
lasso = Lasso(alpha=0.0005, random_state = 1,max_iter=100)

ENet = ElasticNet(alpha = 0.0005, l1_ratio=0.9, random_state = 3,max_iter=100)

GBoost = GradientBoostingRegressor(n_estimators = 100,learning_rate=0.05,
                                   max_depth = 10, random_state=5)

model_rf = RandomForestRegressor(max_depth=17,n_estimators=100)

In [12]:
models = [lasso,ENet,GBoost,model_rf]
scores={}
for model in models:
    print(model)
    model.fit(X_train.values,Y_train.values)
    tr_pred = model.predict(X_test.values)
    print(deviation_metric(Y_test.values,tr_pred))

Lasso(alpha=0.0005, max_iter=100, random_state=1)


C:\Users\gto_n\.conda\envs\raifhack_2021\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.256e+15, tolerance: 8.239e+11
  model = cd_fast.enet_coordinate_descent(
C:\Users\gto_n\.conda\envs\raifhack_2021\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(


4.91867661972802
ElasticNet(alpha=0.0005, l1_ratio=0.9, max_iter=100, random_state=3)


C:\Users\gto_n\.conda\envs\raifhack_2021\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.256e+15, tolerance: 8.239e+11
  model = cd_fast.enet_coordinate_descent(


4.9187240254775295
GradientBoostingRegressor(learning_rate=0.05, max_depth=10, random_state=5)
2.803620569147034
RandomForestRegressor(max_depth=17)


KeyboardInterrupt: 

In [13]:
from sklearn.preprocessing import RobustScaler,StandardScaler,OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error,auc

class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=3):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds

    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)

        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred

        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self

    #Do the predictions of all base models on the test data and use the averaged predictions as
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
import typing
import numpy as np
import catboost as cb

train_dataset = cb.Pool(X_train.values, Y_train)
test_dataset = cb.Pool(X_test.values, Y_test)


class CatBoostEvalMetricRaifhack(object):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        # the larger metric value the better
        return False

    def deviation_metric_one_sample(self, y_pred: typing.Union[float, int], y_true: typing.Union[float, int]) -> float:
        """
        Реализация кастомной метрики для хакатона.

        :param y_true: float, реальная цена
        :param y_pred: float, предсказанная цена
        :return: float, значение метрики
        """
        THRESHOLD = 0.15
        NEGATIVE_WEIGHT = 1.1
        deviation = (y_pred - y_true) / np.maximum(1e-8, y_true)
        if np.abs(deviation) <= THRESHOLD:
            return 0
        elif deviation <= - 4 * THRESHOLD:
            return 9 * NEGATIVE_WEIGHT
        elif deviation < -THRESHOLD:
            return NEGATIVE_WEIGHT * ((deviation / THRESHOLD) + 1) ** 2
        elif deviation < 4 * THRESHOLD:
            return ((deviation / THRESHOLD) - 1) ** 2
        else:
            return 9

    def evaluate(self, y_pred, y_true, weight):
        y_pred = y_pred[0]
        return np.array([self.deviation_metric_one_sample(y_pred[n], y_true[n]) for n in range(len(y_pred))]).mean(), 0


model = cb.CatBoostRegressor(
    iterations=100,
    learning_rate=0.5,
    eval_metric=CatBoostEvalMetricRaifhack(),
    # task_type="GPU",
    # devices='0:1'
)
# model.fit(
#     X_train,
#     Y_train,
#     verbose=False,
#     plot=True
# )

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (model_rf, model),
                                                 meta_model = lasso)


stacked_averaged_models.fit(X_train.values, Y_train.values)

stacked_train_pred = stacked_averaged_models.predict(X_test.values)
print(deviation_metric(Y_test.values, stacked_train_pred))